In [ ]:
!pip install robin_stocks
import robin_stocks.robinhood as r
import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt

Library: [robin_stocks](https://readthedocs.org/projects/robin-stocks/downloads/pdf/latest/)

## Authentication and Login

In [ ]:
login = r.login('kgraghav@mtu.edu','0214@Kgraghavan',store_session=True)

## List of Cryptocurrencies available

In [ ]:
crypto_info=r.crypto.get_crypto_currency_pairs()
crypto_list=[]
for i in range(len(crypto_info)):
    crypto_list.append(crypto_info[i]['asset_currency']['code'])

## Read in Portfolio

In [ ]:
df_portfolio=pd.read_csv('portfolio.csv')
df_portfolio.set_index('Parameters',inplace=True)
columns_list=df_portfolio.columns.tolist()
columns_dict={}
for stock in columns_list:
    stock_str_rep=stock.replace('-','.')
    columns_dict[stock]=stock_str_rep
df_portfolio.rename(columns=columns_dict,inplace=True)
df_portfolio.sort_values(by='weight',axis=1,ascending=False)

In [ ]:
port_stocks=df_portfolio.columns.tolist()

In [ ]:
df_suggested_equity=df_portfolio.loc['suggested_investment',:]
df_suggested_equity.transpose().plot(kind='bar',figsize=(12,8));
total_port_equity=df_suggested_equity.sum()
print('Total suggested equity value is: {}'.format(total_port_equity))


## Print total stocks value

In [ ]:
stocks_dict=r.account.build_holdings()
#print('Stocks are: {}'.format(stocks_dict))
tickers=stocks_dict.keys()
ticker_list=list(tickers)
for ticker in ticker_list:
    ticker.replace('-','.')
print(ticker_list)
###### Get crypto positions #####
crypto_holdings=r.get_crypto_positions()
print(float(crypto_holdings[3]['quantity']))

In [ ]:
price_list=[float(stocks_dict[ticker]['equity']) for ticker in ticker_list]
for i in range(len(crypto_holdings)):
    if float(crypto_holdings[i]['quantity'])>0:
        crypto_ticker=crypto_holdings[i]['currency']['code'].replace('-USD','')
        crypto_qty=float(crypto_holdings[i]['quantity'])
        crypto_mark_price=float(r.get_crypto_quote(symbol=crypto_ticker,info='mark_price'))
        crypto_price=crypto_qty*crypto_mark_price
        price_list.append(crypto_price)
        ticker_list.append('{}.USD'.format(crypto_holdings[i]['currency']['code']))

## Stocks common to portfolio and investment

In [ ]:
comm_stocks=list(set(port_stocks).intersection(set(ticker_list)))
print('Stocks common to recommended and actual portfolio: ',comm_stocks)
port_stocks=[stock.replace('-','.') for stock in port_stocks]

## Stocks not common

### Stock in recommended portfolio but not in actual portfolio

In [ ]:
not_in_ticker_list=[stock for stock in port_stocks if stock not in ticker_list]
not_in_ticker_list

### Stock in  actual portfolio but not in recommended portfolio

In [ ]:
not_in_port_stocks=[ticker for ticker in ticker_list if ticker not in port_stocks]
not_in_port_stocks

## Share Price

In [ ]:
share_price={}
share_price_values=[float(stocks_dict[ticker]['price']) for ticker in tickers]
for i in range(len(tickers)):
    ticker=ticker_list[i]
    share_price[ticker]=share_price_values[i]
share_price=pd.DataFrame(share_price,index=['price'])
share_price

## Share Quantity

In [ ]:
share_quantity={}
share_quantities=[float(stocks_dict[ticker]['quantity']) for ticker in tickers]
for i in range(len(tickers)):
    ticker=ticker_list[i]
    share_quantity[ticker]=share_quantities[i]
share_quantity=pd.DataFrame(share_quantity,index=['quantity'])
share_quantity

## Total Equity

In [ ]:
equity={}
total_equity=0
for i in range(len(ticker_list)):
    ticker=ticker_list[i]
    equity[ticker]=price_list[i]
    total_equity=total_equity+equity[ticker]
print('Total Stocks equity value is: {}'.format(total_equity))

equity_comm={}
total_comm_equity=0
for i in range(len(comm_stocks)):
    ticker=comm_stocks[i]
    equity_comm[ticker]=price_list[i]
    total_comm_equity=total_comm_equity+equity_comm[ticker]
print('Total Common Stocks equity value is: {}'.format(total_comm_equity))

df_equity=pd.DataFrame.from_dict(data=equity,orient='index',columns=['Equity'])
df_plot=df_equity.sort_values(by='Equity',axis=0,ascending=False)
if df_plot.shape[0]>0:
    df_plot.plot(kind='bar',figsize=(12,8))
else:
    print('no data to plot')
df_equity=df_equity.transpose()
df_equity.sort_values(by='Equity',axis=1,ascending=False)

In [ ]:
ticker_list

## Equity Change

In [ ]:
df_suggested_equity

In [ ]:
if len(ticker_list)>0:
    equity_change={}
    total_port_equity_change=0
    for i in range(len(ticker_list)):
        stock=ticker_list[i]
        if stock in df_equity.columns.tolist() and stock in df_suggested_equity.index.tolist():
            equity_change[stock]=df_equity.loc['Equity',stock]-df_suggested_equity[stock]
            total_port_equity_change=total_port_equity_change+equity_change[stock]

    equity_change_percent=total_port_equity_change/total_port_equity*100
    df_equity_change=pd.DataFrame.from_dict(data=equity_change,orient='index',columns=['Equity_Change']).transpose()
    df_equity_change_ratio=pd.DataFrame(df_equity_change.
                                 div(df_suggested_equity))
    df_equity_change_percent=df_equity_change_ratio*100
    df_equity_change_ratio.rename(index={'Equity_Change':'Equity_Change_Ratio'},inplace=True)
    df_equity_change_percent.rename(index={'Equity_Change':'Equity_Change_Percent'},inplace=True)

    if len(comm_stocks)>0:
        equity_comm_change={}
        total_equity_comm_change=0
        for i in range(len(comm_stocks)):
            comm_stock=comm_stocks[i]
            equity_comm_change[comm_stock]=df_equity.loc['Equity',comm_stock]-df_suggested_equity[comm_stock]
            total_equity_comm_change=total_equity_comm_change+equity_comm_change[comm_stock]
        print('Total Common equity change value is: {}'.format(total_equity_comm_change))


        equity_comm_change_percent=total_equity_comm_change/total_port_equity*100
        print('Total Common Change Percent is: {}'.format(equity_comm_change_percent))

        df_equity_comm_change=pd.DataFrame.from_dict(data=equity_comm_change,orient='index',columns=['Equity_Comm_Change']).transpose()
        df_equity_comm_change_ratio=pd.DataFrame(df_equity_comm_change.
                                         div(df_suggested_equity))
        df_equity_comm_change_percent=df_equity_comm_change_ratio*100
        df_equity_comm_change_ratio.rename(index={'Equity_Comm_Change':'Equity_Comm_Change_Ratio'},inplace=True)
        df_equity_comm_change_percent.rename(index={'Equity_Comm_Change':'Equity_Comm_Change_Percent'},inplace=True)
        ############################ Plots #####################################
        df_equity_comm_change.sort_values(by='Equity_Comm_Change',axis=1,ascending=False).transpose().plot(kind='bar',figsize=(18,6));
        df_equity_comm_change_percent.sort_values(by='Equity_Comm_Change_Percent',axis=1,ascending=False).transpose().plot(kind='bar',figsize=(18,6));
    else:
        print('no common stocks')

else:
    print('no positions')

## Update Stop Loss

### Stop Loss Sell Limit Price

In [ ]:
stop_loss_sell_limit_price={}
if len(comm_stocks)>0:
    for i in range(len(comm_stocks)):
        stock=comm_stocks[i]
        try:
            stop_loss_sell_diff=df_portfolio.loc['price_bound',stock]
            limit_price=float(share_price[stock]-stop_loss_sell_diff)
            if limit_price<0:
                stop_loss_sell_limit_price[stock]=0
            else:
                stop_loss_sell_limit_price[stock]=limit_price
        except:
            print(stock)
else:
    print('no common stocks')
stop_loss_sell_limit_price

### Cancel Existing Stock Orders

### Update Stop Loss Sell orders for all stocks

## Sell and Buy Stocks

### Sell all portfolio stocks

### Sell all stocks

### Buy Portfolio Stocks

#### 